# NLP classification Modeling Function Library

This notebook contains a copy of all of the functions we constructed and use in this porject. What the documentation does not explain for each function, we hope to elaborate in here.

In [17]:
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from sklearn.feature_extraction import text
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

___

# Api Request's

___

https://github.com/pushshift/api 

We used Pushift API which is run on a small pirvate server. As such we modified our API request make small recursive pulls spread out over time to avoi clogging the network or recieving a ban.

In [2]:
def get_posts(strokes, subreddit, target_time):
    df = pd.DataFrame()
    master_df = pd.DataFrame()
    
   
    for i in range(strokes):
        try:
            # set the parameters for 
            params = {
                'subreddit': subreddit,
                'size': 50,
                'before': target_time}

            res = requests.get(url,params)
            data = res.json()
            posts = data['data']
            df = pd.DataFrame(posts)

            frames = [df,master_df]
            master_df = pd.concat(frames, axis= 0 , ignore_index = True)

            target_time = df['created_utc'].min()

            time.sleep(10)
        except KeyError:
            continue

    
# for when created_utc does not exist     

       
    return master_df


In [3]:
# Creates dataframe from an HTML request response
def res_to_df(res):
    ''' input: html request results  
        output: dataframe
    '''
    data = res.json()
    posts = data['data']
    return pd.DataFrame(posts)


___

# Creating Visualizations

___

In [4]:
def disp_hud(hud):
    
    """Useful Hud for checking our tokenization and datframe concatonations work well"""
   
    # here we create tables we want to display 
    base_group = corpus.groupby(['fact']).mean()
    head = corpus.head(2)
    
    # here we list them and pair them with names
    hud = [base_group,head]
    disp = ['mean','preview']
    
    # iterate through list of tables 
    # print name and displauy table
    for i,li in enumerate(hud):
        print(disp[i])
        display(li)

Here we have a function for plotting our own custom confusion matrix

In [ ]:
# custom confusion matrix plot

def disp_cm(model,X,y):
    
    """
    model: model name
    X: train or test features list
    """
    preds = model.predict(X)
   
    cm = confusion_matrix(y, preds)
    
    fig  = ConfusionMatrixDisplay(cm).plot();
    fig = plt.title(f'{model[1]} \n MSE: {mean_squared_error(y ,preds)}')
    fig = plt.xlabel((('Fiction'),('Fact')))
    
    return fig

___

# Tokenizing Text 

___

We created a series of functions for tokeninzing and modifying our datafarame. Here you will find find three steps which are then combined into a fourth which calls the prior three. 

1). Use Regex Tokenizer to remove punction and whitespace in tokenized output of textual input.

In [5]:
# tokenize the text field of each column and remove pnctuation.

def make_tokens(corpus,column):    
    
    '''
    corpus: dataframe with field to tokenize
    column: the name of the column you would like to tokenize
    
    output: dataframe with tokenized text field
    '''
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    corpus[column] = [tokenizer.tokenize(row) for row in corpus[column]]
    return corpus

2). Remove tokens from coprus with using text.ENGLISH_STOP_WORDS and custom stop word list 

In [6]:
def clean_tokens(corpus,column):    
    '''
    corpus: dataframe with tokenized text field
    column : text column
    
    output: dataframe with stopword tokens removed
    '''
    
    # custom stop word list
    my_stop_words = ['https','com','www','people','know','actually',
                     'world','time','years','fact','facts','fake','like',
                     'sk','10','en','day','water','did','just','the']
    
    # append custom stopwords to text.ENGLIS_STOP_WORDS
    stop_words = text.ENGLISH_STOP_WORDS.union(my_stop_words)
    
    # CLean rows of words in stop words list
    for row in corpus[column]:
        for word in row:
            if word in stop_words:
                row.remove(word)
    
    # returns clean datframe.
    return corpus

Depending on the circumstance you may want to perform make_tokens & clean_tokens without running the following function string_tokens so that you can maintain a dataframe with a tokekenized text field. 

In [19]:
def string_tokens(corpus,column):
    '''    
    corpus: dataframe with tokenized text field
    column: text column
    
    output: dataframe with string for text field
    '''
    
    # returns tokinzed columns back to strings
    for i , row in enumerate(corpus[column]):
        corpus[column][i]=' '.join(corpus[column][i])
    # returns clean strings df
    return corpus

In [18]:
def make_clean_string_corpus(corpus,column):
    
    '''
    runs eac of three functions and outputs a dataframe where 
    the text field has had punctuation and stop words removed removed &
    haas been returned to a string to a string.
   
    '''
    # runs each of the three prior functions and returns  
    corpus = make_tokens(corpus,column)
    corpus = clean_tokens(corpus, column)
    corpus = string_tokens(corpus,column)
            
    return corpus

___

# Model Pipelines 

___

Now we has a series of functions created for the fitting and scoreing of numerous pieplines.

example list of pipelines requiered to run these functions

pipelines = [

    ("ModelName_1", pipeline_1),
    
    ("ModelName_2", pipeline_2),

    ]

In [9]:
def fit_pipes(pipelines,X,y):
    """
     fits a model for each pipeline in our pipelines list
    """
    for name, model in pipelines:
        model.fit(X,y)

In [15]:
def score_pipes(pipelines,X,y):
    """
    score all pipes in pipelines
    pipelines: list of tuples
    
    X = features of population to score .. ie ( X_train )
    y = target feature population to score .. ie ( y_train )
    """
    scores = []
    df = pd.DataFrame()

    # for each name and model in pipelines
    for name, model in pipelines:
        # score the given population 
        scores.append(model.score(X,y))
        #print the madel name and score
        print( f'{name} Accuracy: {model.score(X,y)} ' )
    #retunrs all scores in a datframe.

    df['Scores'] = scores
    df['Model'] = [name for name, model in pipelines]
    df.set_index('Model')
    
    return df

For model selection assesing the varriance in Train and test Scores is made easier by visualizing the gap between them among models.

In [20]:
def get_gap(pipelines,train_scores,test_scores):
    ''' 
    for assesing the gap between train and test model scores
    
    pipelines: list fo tuples
    scores: lists
    
    output: dataframe
    '''
    output = []
    
    for i , li in enumerate(train_scores):
        dif =  li - test_scores[i]
        output.append(dif)
    result = pd.DataFrame()
    result['Model'] = [name for name , model in pipelines]
    result['Gap'] = [li for li in output]
    
    return result

In [13]:

def best_score(pipelines,X,y):
    """
    returns the name and score of the model with the highest test score
    
    pipelines: list of tuples
    
    X = features of population to score .. ie ( X_train )
    y = target feature population to score .. ie ( y_train )
    
    """
    
    best_accuracy = 0.0
    best_classifier = '' 
    best_pipeline = ''
    
    
    
    for name, model in pipelines:
        if model.score(X,y) > best_accuracy:
            best_accuracy = model.score(X, y)
            best_pipeline = model
            best_classifier = name
            
    print(f' Best Accuracy: {best_accuracy}')
    print(f'Model: {best_classifier}')

This makes it easy for us to concatonate the results of our 3 modle test scores metrics into one datframe. 

In [16]:
def make_results_df(pipelines,X_train,y_train,X_test,y_test):
    
    '''
    This function scores test and train population for all models in pipelines list
    The Gap is calculated by taking the difference in scores
    A dataframe is returned with train, test , and gap values for each model.
    '''
    
    train_scores = score_pipes(pipelines,X_train,y_train)
    test_scores = score_pipes(pipelines,X_test,y_test)
    output = pd.merge(left = train_scores,right =  test_scores, left_on = 'Model',right_on = 'Model')
    train_score_list = [ score for score in train_scores['Scores']]
    test_score_list = [ score for score in test_scores['Scores']]
    gap_results = get_gap(pipelines,train_score_list,test_score_list)
    output = pd.merge(output, gap_results, left_on='Model', right_on='Model')
    output.set_index('Model')
    return output
    
    